# **Lane Lines Detection Using Python and OpenCV**

**In this project, I have detected lane lines on the road using _Python_ and _OpenCV_.**

**I have developed a computer vision pipeline that processes a group of test images then applied this pipeline to two test video streams.**

## Pipeline Architecture:

1. Load test images
2. Apply grayscale transform
3. Smooth the image to suppress noise and any spurious gradients
4. Apply Canny edge detection algorithm
5. Cut-out any edges that are out of the lane lines region (**region of interest**)
6. Get the lines located in the region of interest using hough transform
7. Fitting and extrapolating these lines for both right and left lane
  * Determining the x and y points of each of the right and left lane lines
  * Extrapolating or fitting these points for both lane lines
8. Draw the lane lines on the original image
  * Drawing the lane lines on a blank image firstly
  * Weighting the original image with the lane lines image

## Environment:
* Ubuntu 16.04 LTS
* Python 3.6.4
* OpenCV 3.1.0
* Anaconda 4.4.10

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

## 1. Load test images

The test images are loaded to a list `test_images` which will be used to feed our pipeline which test images

In [2]:
images_list = os.listdir("test_images/")
test_images = []
for img in images_list:
    test_images.append(mpimg.imread("test_images/"+img))

## 2. Apply grayscale transform

In [3]:
def grayscale(image):
    """
    Description:
        Applies the gray-scale transform
    Parameters:
        image: A color input image
    Output:
        A gray-scaled image
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

## 3. Smooth the image to suppress noise and any spurious gradients

In [4]:
def gaussian_blur(image, kernel_size = 5):
    """
    Description:
        Smoothes the image by applying guassian filter to the image
    Parameters:
        image: A gray-scaled input image
        kernel_size (Default = 5): This is the window the convolves the whole image applying the filter to the image
    Output:
        Smoothed (Averaged or Filtered) image
    """
    
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

## 4. Apply Canny edge detection algorithm

**Wikipedia**

The Canny edge detector is an edge detection operator that uses a multi-stage algorithm to detect a wide range of edges in images.

**Process of Canny edge detection algorithm**

1. Apply Gaussian filter to smooth the image in order to remove the noise
2. Find the intensity gradients of the image
3. Apply non-maximum suppression to get rid of spurious response to edge detection
4. Apply double threshold to determine potential edges
5. Track edge by hysteresis: Finalize the detection of edges by suppressing all the other edges that are weak and not connected to strong edges.


In [5]:
def canny(image, low_threshold  = 50, high_threshold = 150):
    """
    Description:
        Canny edge detector which detects strong edges (strong gradients pixels) above the high threshold and 
        rejects pixels below the low threshold. Pixels with values between low and high thresholds will be included
        as long as they are connected to strong edges
    Parameters:
        image: The input gray-scaled smoothed image
        low_thresold (Default = 50): The threshold  value of rejecting pixel
        high_threshold (Default = 150): The threshold of strong edges in the image
    Output:
        A binary image with pixels tracing out the detected edges and black everything else
    """
    
    return cv2.Canny(image, low_threshold, high_threshold)

## 5. Cut-out any edges that are out of the lane lines region (region of interest)

In [6]:
def region_of_interest(image):
    """
    Descrition:
        Applies an image mask.Only keeps the region of the image defined by the polygon
        formed from `vertices`. The rest of the image is set to black.
    Paramteres:
        image: The image from Canny edge detector
    Output:
        An image that has the edges in the defined polygon and black everywhere else
    """
    
    image_shape  = image.shape
    # The following points are the vertices points of the polygon
    bottom_left  = (110,image_shape[0])
    up_left      = (450, image_shape[0]/1.65)
    bottom_right = (image_shape[1]-450, image_shape[0]/1.65)
    up_right     = (image_shape[1]-20,image_shape[0])
    vertices     = np.array([[bottom_left, up_left, bottom_right, up_right]], dtype=np.int32)
    
    #defining a blank mask to start with
    mask = np.zeros_like(image)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

## 6. Get the lines located in the region of interest using hough transform

In [7]:
def hough_transform(image):
    """
    Descrition:
        Hough Transfrom is used to indetify lines located in the region of interest
    Parameters:
        image: This is the output a Canny transform and after applying the region of interest mask.
    Output:
        Returns a list with hough lines.
    """
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = (np.pi/180) # angular resolution in radians of the Hough grid
    threshold = 10     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 1 #minimum number of pixels making up a line
    max_line_gap = 1    # maximum gap in pixels between connectable line segments
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), minLineLength=min_line_length, maxLineGap=max_line_gap)
    return lines

## 7. Fitting and extrapolating these lines for both right and left lane

This step is done into two mian steps:
* Determining the x and y points of each the right and left lane lines
* Extrapolating or fitting these points for both lane lines

### Determining the x and y points of the right lane

In [8]:
def get_right_lane_points(lines_image):
    """
    Descrition:
        This function calculates the slope of each line detected from hough transform. 
        If the slope is +ve, then this line belongs to the right lane.
        Note: 0.4 is selected for better filteration of the lines ouput
    Parameters:
        lines_image: This is the output of hough tranfrom (a list contains the hough lines)
    Output
        right_lane_x_points: a list contains the all x points of the right lane
        right_lane_y_points: a list contains the all y points of the right lane
    """
    
    right_lane_x_points = []
    right_lane_y_points = []
    
    for line in lines_image:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if(slope > 0.4): #+ve slope -> right lane
                #print("Right Lane Detected\n")
                right_lane_x_points.append(x1)
                right_lane_x_points.append(x2)
                right_lane_y_points.append(y1)
                right_lane_y_points.append(y2)
                
    return right_lane_x_points, right_lane_y_points

### Determining the x and y points of the left lane

In [9]:
def get_left_lane_points(lines_image):
    """
    Descrition:
        This function calculates the slope of each line detected from hough transform. 
        If the slope is -ve, then this line belongs to the left lane.
        Note: -0.6 is selected for better filteration of the lines ouput
    Parameters:
        lines_image: This is the output of hough tranfrom (a list contains the hough lines)
    Output
        left_lane_x_points: a list contains the all x points of the left lane
        left_lane_y_points: a list contains the all y points of the left lane
    """
    
    left_lane_x_points  = []
    left_lane_y_points  = []

    for line in lines_image:
        for x1,y1,x2,y2 in line:
            slope = (y2 - y1) / (x2 - x1)
            if(slope < -0.6): #-ve slope -> left lane
                #print("Left Lane Detected\n")
                left_lane_x_points.append(x1)
                left_lane_x_points.append(x2)
                left_lane_y_points.append(y1)
                left_lane_y_points.append(y2)
                    
    return left_lane_x_points, left_lane_y_points

### Extrapolating or fitting the points for both lane lines

In [10]:
def fit_lane_line(lane_line_x_points, lane_line_y_points, image_shape):
    """
    Descrition:
        The function extrapolates the detectioned points in each lane from the line equation given that
        Ymin and Ymax are defined from the region of interest.
        IF:   y = mx + b
        THEN: x = (y - b)/m
    Parameters:
        lane_line_x_points: This is a list for the x points of our intended lane line
        lane_line_y_points: This is a list for the y points of our intended lane line
        image_shape: The height and width of the image to get the Ymin and Ymax
        
    Output
        Xmin, Ymin: The start point of our intended lane line
        Xmax, Ymax: The end point of our intended lane line
    """
    
    #Getting the slop and intersect of the right lane line
    fit_lane_line =  np.polyfit(lane_line_x_points, lane_line_y_points, 1)
    m = fit_lane_line[0] # Slope
    b = fit_lane_line[1] # Intercept

    #These are the y values defined at my region of interest
    Ymax = image_shape[0]
    Ymin = image_shape[0]/1.65
    
    # If equation of a line: y = m*x + b
    # Hence: x = (y - b)/m
    Xmax =  (Ymax - b) / m
    Xmin =  (Ymin - b) / m

    #Converting the values from float to int to be suitable for cv2.line
    Ymax = int(Ymax)
    Ymin = int(Ymin)
    Xmax = int(Xmax)
    Xmin = int(Xmin)
    
    return Xmax, Xmin, Ymax, Ymin

## 8. Draw the lane lines on the original image

In this step, we are:
* Drawing the lane lines on a blank image firstly
* Weighting the original image with the lane lines image

### Drawing right and left lane lines on blank image

In [11]:
def draw_lines(image, Xmin, Xmax, Ymin, Ymax):
    """
    Descrition:
        This function is used to draw the lane line on a blank image       
    Parameters:
        Xmin, Ymin: The start point of our intended lane line
        Xmax, Ymax: The end point of our intended lane line
    Output:
        Image of the lane line drawn on it
    """
    
    #Drawing the lane line on the blank image
    return cv2.line(image,(Xmin,Ymin),(Xmax,Ymax),(255,0,0),10)

### Weighting the original image with the lane lines image

In [12]:
def weighted_img(image, initial_img, α=0.8, β=1., γ=0.):
    """
    Descrition:
        This function is used to weight (combine) the lane lines image and the original one
    Parameters:
        image: The image which has the lane lines
        initial_img: should be the image before any processing.
    Output:
        image is computed as follows: initial_img * α + image * β + γ
        
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, image, β, γ)

## Here we save the image after the pipeline for the specified path ```test_images_output```

In [13]:
def save_image(image, test_image_index):
    """
    Descrition:
        This function is save an image to the specified path with the same origianal name
    Parameters:
        image: should be the image after the pipeline
        test_image_index: an index to retrieve the name of the test image under processing
    Output:
        Saving the image in the test_images_output directory with the same original name
    """
    
    path = 'test_images_output/'
    cv2.imwrite(os.path.join(path , images_list[test_image_index]), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

## Process Pipeline:
This is our algorithm pipeline entry point.
This function wraps and calls all the functions needed to detect the lane lines on the road of an image

In [14]:
def process_pipeline(image):
    """
    Descrition:
        This function is save an image to the specified path with the same origianal name
    Parameters:
        image: image under test to detect the lane lines in it
    Output:
        result: The output of our pipeline. It should be the original image with the lane lines annotated.
    """
    image_shape  = image.shape

    gray_scale_image = grayscale(image)
    
    smoothed_gray_scale_image = gaussian_blur(gray_scale_image)
    
    edge_detected_image = canny(smoothed_gray_scale_image)
    
    masked_edge_detected_image = region_of_interest(edge_detected_image)
    
    lines_image = hough_transform(masked_edge_detected_image)
    
    right_lane_x_points , right_lane_y_points = get_right_lane_points(lines_image)
    left_lane_x_points  , left_lane_y_points  = get_left_lane_points(lines_image)
    
    Xright_min , Xright_max , Yright_min , Yright_max = fit_lane_line(right_lane_x_points, right_lane_y_points, image_shape)
    Xleft_min  , Xleft_max  , Yleft_min  , Yleft_max  = fit_lane_line(left_lane_x_points, left_lane_y_points, image_shape)
    
    
    lane_lines_image = np.copy(image)*0  # creating a blank to draw lines on
    lane_lines_image = draw_lines(lane_lines_image, Xright_min , Xright_max , Yright_min , Yright_max)
    lane_lines_image = draw_lines(lane_lines_image, Xleft_min  , Xleft_max  , Yleft_min  , Yleft_max)
    
    result = weighted_img(lane_lines_image, image)
    
    return result

## Test on Test Images

In [15]:
for i , img in enumerate(test_images, start = 0):
    result = process_pipeline(img)
    save_image(result, i)

The test images should now be saved at `test_images_output` folder

## Test on Videos

In [16]:
def process_image(image):
    
    result = process_pipeline(image)
    
    return result

## Test solidWhiteRight.mp4

In [17]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:07<00:00, 30.10it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 6.21 s, sys: 225 ms, total: 6.43 s
Wall time: 8.35 s


The outout video should be saved at `test_videos_output` and the file named by `solidWhiteRight.mp4`

## Test solidYellowLeft.mp4

In [18]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:25<00:00, 26.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 19.7 s, sys: 657 ms, total: 20.3 s
Wall time: 26.8 s


The outout video should be saved at `test_videos_output` folder and the file named by `solidYellowLeft.mp4`